<a href="https://colab.research.google.com/github/VictorSilvaCamargo/BigData/blob/main/PySpark_Censo_Escolar_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("PySpark").master('local[*]').getOrCreate()


sc = spark.sparkContext

In [4]:
from pyspark import SparkContext


rdd = sc.textFile("censo_escolar_2021.csv")


rdd_filtered = rdd.map(lambda x: x.lower().replace(".","").replace(",",""))


rdd_filtered = rdd_filtered.filter(lambda x:x.split(";")[0] != "nu_ano_censo")


In [5]:

rdd_curitiba = rdd_filtered.filter(lambda x: x.split(";")[6] == "curitiba")


total_escolas = rdd_curitiba.count()


with open('Escolas_Curitiba.txt', 'w') as f:
    f.write(str(total_escolas) + '\n')


In [6]:
from functools import reduce

rdd_chaveValor = rdd_filtered.map(lambda x: (x.split(";")[1], 1))

rdd_count = rdd_chaveValor.reduceByKey(lambda x, y: x + y)

rdd_escolaPorRegiao = rdd_count.sortBy(lambda x: x[0], ascending=True)

rdd_escolaPorRegiao.collect()

dados = rdd_escolaPorRegiao.collect()

with open('Escolas_Por_Regiao.txt', 'w') as f:
    for linha in dados:
        f.write(str(linha) + '\n')


In [7]:
def build_tuple(s):
    fields = s.split(";")
    qt_doc_bas = int(fields[338]) if fields[338] else 0
    qt_doc_fund = int(fields[342]) if fields[342] else 0
    qt_doc_med = int(fields[345]) if fields[345] else 0
    return ((fields[14], fields[6]),(qt_doc_bas + qt_doc_fund + qt_doc_med))


rdd_escola = rdd_filtered.map(build_tuple)
rdd_escola_sorted = rdd_escola.sortBy(lambda x: x[1], ascending=False)

rdd_escola_sorted.take(1)

primeira_linha = rdd_escola_sorted.first()

with open('Primeira_Linha.txt', 'w') as f:
    f.write(str(primeira_linha))

In [8]:
from functools import reduce

def build_tuple(s):
  fields = s.split(";")
  qt_mat_med = int(fields[305]) if fields[305] else 0
  return (fields[1], (qt_mat_med, 1))

rdd_medio = rdd_filtered.map(build_tuple)
rdd_medio_media = rdd_medio.reduceByKey(lambda x,y:(x[0] + y[0], x[1] + y[1]))
rdd_medio_media = rdd_medio_media.map(lambda x:(x[0], x[1][0] / x[1][1]))
rdd_medio_media.collect()

rdd_dados = rdd_medio_media.collect()

with open('Media_Matriculas.txt', 'w') as f:
    for linha in rdd_dados:
        f.write(str(linha) + '\n')

In [9]:
from functools import reduce

def build_tuple(s):
  fields = s.split(";")
  tp_localizacao = int(fields[17]) if fields[17] else 0
  tp_dependendia = int(fields[15]) if fields[15] else 0
  return ((tp_localizacao, tp_dependendia), 1)

rdd_escola_por_localizacao = rdd_filtered.map(build_tuple)

rdd_teste = rdd_escola_por_localizacao.reduceByKey(lambda x, y: x + y)
rdd_teste = rdd_teste.sortByKey()

rdd_teste.collect()

rdd_dados = rdd_teste.collect()


with open('Quantidade_Por_Localização_Dependencia.txt', 'w') as f:
    for linha in rdd_dados:
        f.write(str(linha) + '\n')


In [10]:
from functools import reduce

def build_tuple(s):
  fields = s.split(";")
  sg_uf = fields[4]
  tp_dependendia = int(fields[15]) if fields[15] else 0
  return ((sg_uf, tp_dependendia), 1)

rdd_quantidade = rdd_filtered.map(build_tuple)
rdd_quantidade_re = rdd_quantidade.reduceByKey(lambda x,y: x + y)
rdd_ordenado = rdd_quantidade_re.sortBy(lambda x: x[1], ascending=False)


rdd_ordenado.collect()
rdd_dados = rdd_ordenado.collect()


with open('Quantidade_Por_UF_Dependencia.txt', 'w') as f:
    for linha in rdd_dados:
        f.write(str(linha) + '\n')